In [19]:
# Do all relevant imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
import time
import scipy
import tables as tb

In [83]:
class Matrix_multiplicator(object): 
    def __init__(self):
        self.bl = 500 #this is the number of rows we calculate each loop
    
    def dot(self, m1, m2, filename, result_format = 'csr'): 
        starttime = time.time()
        
        l, m, n = m1.shape[0], m1.shape[1], m2.shape[1]
        
        self.f = tb.open_file(filename, 'w')
        filters = tb.Filters(complevel=5, complib='blosc')
        
        out_data = self.f.create_earray(self.f.root, 'data', tb.Float32Atom(), shape=(0,), filters=filters)
        out_indices = self.f.create_earray(self.f.root, 'indices', tb.Int64Atom(),shape=(0,), filters=filters)
        out_indptr = self.f.create_earray(self.f.root, 'indptr', tb.Int64Atom(), shape=(0,), filters=filters)
        out_data.append(np.zeros(m1.shape[0]*m2.shape[1],))
        out_indices.append(np.zeros(m1.shape[0]*m2.shape[1],))
        out_indptr.append(np.zeros(m1.shape[0]*m2.shape[1],))
        

        out_indptr.append(np.array([0])) #this is needed as a first indptr
        max_indptr = 0
        print("Heading into loop. %s sec"%(time.time()-starttime))
        cp = time.time()
        count = 0
        for i in range(0, l, self.bl):
            if i % 5000 == 0 or i == 500: 
                print("Row %s out of %s, %s min."%(i, l, (time.time()-cp)/60))
                cp = time.time()
            res = m1[i:min(i+self.bl, l),:].dot(m2)
            #out_data.append(res.data)
            out_data[count:count+len(res.data)]
            indices = res.indices
            indptr = res.indptr
            #out_indices.append(indices)
            out_indices[count:count+len(indices)] = indices
            #out_indptr.append(max_indptr+indptr[1:])
            out_indptr[count:count+len(indptr)] = indptr
            max_indptr += indices.shape[0]
            count += len(res.data)
            
        if result_format == 'csr': 
            m = sps.csr_matrix((self.f.root.data[:], self.f.root.indices[:], self.f.root.indptr[:l+1]), shape=(l,n))
        elif result_format == 'csc': 
            m = sps.csc_matrix((self.f.root.data[:], self.f.root.indices[:], self.f.root.indptr[:l+1]), shape=(l,n))
        
        self.f.close()
        print("Calculated m.dot() to %s. %s sec" %(filename, time.time()-starttime))
        
    def get_result(self, filename, l, n, result_format = 'csr'):
        self.h5 = tb.open_file(filename, 'r')
        if result_format == 'csr':
            m = sps.csr_matrix((self.h5.root.data[:], self.h5.root.indices[:], self.h5.root.indptr[:l+1]), shape=(l,n))
        elif result_format == 'csc': 
            m = sps.csc_matrix((self.h5.root.data[:], self.h5.root.indices[:], self.h5.root.indptr[:l+1]), shape=(l,n))
        self.h5.close()
        return m
m = Matrix_multiplicator()

In [84]:
a = sps.rand(50, 100).tocsr()
b = sps.rand(100, 200).tocsc()

In [85]:
#m.f.close()

In [89]:
#m.dot(a,b,'h5/tes7.h5')
c = (m.get_result('h5/tes7.h5', 50, 200))
print(a.nnz)
print(b.nnz)
print(c.nnz)
print(c)

50
200
90
  (0, 153)	0.0
  (0, 63)	0.0
  (0, 22)	0.0
  (1, 46)	0.0
  (2, 165)	0.0
  (2, 80)	0.0
  (3, 94)	0.0
  (5, 169)	0.0
  (5, 86)	0.0
  (12, 124)	0.0
  (13, 22)	0.0
  (15, 166)	0.0
  (15, 41)	0.0
  (15, 191)	0.0
  (15, 162)	0.0
  (15, 10)	0.0
  (16, 90)	0.0
  (19, 46)	0.0
  (21, 141)	0.0
  (21, 115)	0.0
  (21, 77)	0.0
  (21, 47)	0.0
  (22, 185)	0.0
  (23, 139)	0.0
  (23, 5)	0.0
  :	:
  (38, 85)	0.0
  (39, 156)	0.0
  (39, 101)	0.0
  (39, 46)	0.0
  (39, 9)	0.0
  (42, 85)	0.0
  (42, 47)	0.0
  (43, 156)	0.0
  (43, 72)	0.0
  (43, 14)	0.0
  (43, 169)	0.0
  (43, 86)	0.0
  (45, 126)	0.0
  (45, 123)	0.0
  (45, 86)	0.0
  (46, 155)	0.0
  (48, 131)	0.0
  (48, 18)	0.0
  (49, 115)	0.0
  (49, 77)	0.0
  (49, 163)	0.0
  (49, 88)	0.0
  (49, 60)	0.0
  (49, 46)	0.0
  (49, 8)	0.0
